In [6]:
# Custom utilities
import NO_utils_multiple
import NO_utils

# Standard libraries
import sys
import os
import time
import random
from datetime import datetime

# Scientific computing
import numpy as np
import scipy as sp

# Deep learning - PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.fft
import torch.optim as optim
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import (
    DataLoader, Dataset, TensorDataset,
    random_split, ConcatDataset
)
from torchvision import transforms

# Neural operator specific
from neuralop.models import FNO, FNO2d, TFNO, TFNO2d
from neuralop import Trainer, LpLoss, H1Loss

# Data processing and visualization
from sklearn.preprocessing import StandardScaler
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# Progress tracking
from tqdm import tqdm

# Force garbage collection to release file handles
import gc
gc.collect()

In [ ]:
current_dir = os.getcwd()
reserved_dataset_path = os.path.join(current_dir, 'data', 'reduced', 'set reserved', 'reserved_dataset1_ws4_bs2_f32.pt')
reserved_dataset = torch.load(reserved_dataset_path)

print("Reserved Dataset :")
print(f"Type: {type(reserved_dataset)}")
print(f"Length: {len(reserved_dataset)}")
if hasattr(reserved_dataset, 'tensors'):
    for i, tensor in enumerate(reserved_dataset.tensors):
        print(f"Tensor {i}: shape {tensor.shape}, dtype {tensor.dtype}")


# Get info about first sample to understand structure
first_sample = reserved_dataset[0]
if isinstance(first_sample, tuple):
    print("\nFirst sample structure:")
    for i, item in enumerate(first_sample):
        if torch.is_tensor(item):
            print(f"Item {i}: shape {item.shape}, dtype {item.dtype}")
        else:
            print(f"Item {i}: type {type(item)}")

In [ ]:
def visualize_sample(input_tensor, output_tensor):
    """
    Visualize input and output tensors from a single sample.
    
    Args:
        input_tensor: Tensor of shape (3, H, W) containing input components
        output_tensor: Tensor of shape (4, H, W) containing output components
    """
    # Create figure for input components
    fig1 = plt.figure(figsize=(16, 4))

    # Plot input tensor components (1x3 subplot)
    for i in range(3):
        plt.subplot(1, 3, i+1)
        im = plt.imshow(input_tensor[i].abs().numpy())
        plt.colorbar(im)
        plt.title(f'Input Component {i+1}')

    plt.tight_layout()
    plt.show()

    # Create figure for output components 
    fig2 = plt.figure(figsize=(16, 4))

    # Plot output tensor components (1x4 subplot)
    for i in range(4):
        plt.subplot(1, 4, i+1)
        im = plt.imshow(output_tensor[i].abs().numpy())
        plt.colorbar(im)
        plt.title(f'Output Component {i+1}')

    plt.tight_layout()
    plt.show()

In [ ]:
# Get a random sample from the dataset
random_idx = np.random.randint(0, len(reserved_dataset))
sample = reserved_dataset[random_idx]

# Extract input and output tensors from the sample
input_tensor = sample[0]  # First element is input
output_tensor = sample[1] # Second element is output

print(f"Visualizing random sample {random_idx}")
visualize_sample(input_tensor, output_tensor)

In [ ]:
def evaluate_models(model_paths):
    """
    Evaluate multiple models on the dataset and report average L1 error for each.
    
    Args:
        model_paths: List of relative paths to model weight files
    """
    # Store results
    results = {}
    
    for model_path in model_paths:
        print(f"\nEvaluating model: {model_path}")
        
        # Load model weights
        state_dict = torch.load(model_path, map_location=torch.device('cpu'))
        
        # Initialize new model and load weights
        model = NONet()
        model.load_state_dict(state_dict)
        model.eval()
        
        # Calculate average L1 error across dataset
        total_l1_error = 0.0
        num_samples = len(reserved_dataset)
        
        with torch.no_grad():
            for sample in reserved_dataset:
                input_tensor = sample[0]
                target_tensor = sample[1]
                
                # Get model prediction
                pred_tensor = model(input_tensor.unsqueeze(0)).squeeze(0)
                
                # Calculate L1 error
                l1_error = torch.mean(torch.abs(pred_tensor - target_tensor))
                total_l1_error += l1_error.item()
        
        # Store average error
        avg_l1_error = total_l1_error / num_samples
        results[model_path] = avg_l1_error
        print(f"Average L1 Error: {avg_l1_error:.6f}")
    
    return results

# Example usage:
model_paths = [
    'model_weights/RAdam_20241113_004451/RAdam_epoch_3_lr=0.01_weight_decay=0.pt',
    # Add more model paths here
]
results = evaluate_models(model_paths)


In [ ]:
# Example usage
plot_model_weights_histogram('model_weights/RAdam_20241113_004451/RAdam_epoch_3_lr=0.01_weight_decay=0.pt')

C:\Users\hz283\AppData\Local\Temp\ipykernel_60800\2814157227.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location=torch.devic